In [5]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import sys

# Determine root directory - go up from notebooks folder to portfolio_analyser
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from full_scanner import (
    get_sp500_tickers,
    get_nasdaq100_tickers,
    get_portfolio_tickers,
    scan_stocks,
    filter_buy_signals,
    filter_sell_signals,
    create_excel_output,
    create_portfolio_excel,
    create_pdf_report
)

# Setup results directory
results_dir = ROOT / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("✓ Scanner modules loaded")
print(f"✓ Results directory: {results_dir}")
print(f"✓ Timestamp: {timestamp}")

✓ Scanner modules loaded
✓ Results directory: c:\workspace\portfolio_analyser\scanner_results
✓ Timestamp: 20260108_2003


## Configuration

Set your scan parameters:
- **daily_bars**: Number of daily bars to analyze (default: 60)
- **weekly_bars**: Number of weekly bars to analyze (default: 52)
- **concurrency**: Number of parallel workers (default: 2, max recommended: 5)

In [6]:
# Scan parameters
DAILY_BARS = 60
WEEKLY_BARS = 52
CONCURRENCY = 2  # Be careful with higher values (rate limits)

print(f"Daily bars: {DAILY_BARS}")
print(f"Weekly bars: {WEEKLY_BARS}")
print(f"Concurrency: {CONCURRENCY}")

Daily bars: 60
Weekly bars: 52
Concurrency: 2


## Scan S&P 500

Full scan of ~500 stocks. Takes 5-10 minutes depending on cache. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [ ]:
print("Loading portfolio tickers from stocks.txt...")
portfolio_tickers = get_portfolio_tickers()

if portfolio_tickers:
    print(f"Found {len(portfolio_tickers)} portfolio stocks\n")

    # Run scan
    portfolio_results = scan_stocks(
        portfolio_tickers,
        category="Portfolio",
        daily_bars=DAILY_BARS,
        weekly_bars=WEEKLY_BARS,
        concurrency=CONCURRENCY
    )

    if not portfolio_results.empty:
        # Get buy signals (all and quality-filtered)
        portfolio_buy_all = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD', quality_filter=False)
        portfolio_buy = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD', quality_filter=True)
        
        # Get sell signals (all bearish signals)
        portfolio_sell = filter_sell_signals(portfolio_results, quality_filter=False)
        
        # Display summary
        print(f"\n📊 Portfolio Results:")
        print(f"   ✅ {len(portfolio_buy)} buy signals (EXCELLENT/GOOD/OK)")
        if len(portfolio_buy_all) > len(portfolio_buy):
            print(f"   ⏸️  {len(portfolio_buy_all) - len(portfolio_buy)} extended/weak (wait)")
        print(f"   🔴 {len(portfolio_sell)} sell signals (bearish)")
        
        # Display buy opportunities
        if not portfolio_buy.empty:
            print("\n✅ BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
            display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
            if all(col in portfolio_buy.columns for col in display_cols):
                print(portfolio_buy[display_cols].to_string(index=False))
            else:
                print(portfolio_buy[['ticker', 'signal', 'current_price']].to_string(index=False))
        
        # Display sell signals
        if not portfolio_sell.empty:
            print("\n🔴 SELL SIGNALS (Bearish - reduce exposure):")
            display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
            if all(col in portfolio_sell.columns for col in display_cols):
                print(portfolio_sell[display_cols].to_string(index=False))
            else:
                print(portfolio_sell[['ticker', 'signal', 'current_price']].to_string(index=False))
        
        # Save combined results
        if not portfolio_buy.empty or not portfolio_sell.empty:
            xlsx_path = results_dir / f'portfolio_scanner_{timestamp}.xlsx'
            pdf_path = results_dir / f'scanner_report_portfolio_{timestamp}.pdf'
            
            # Create reports with both buy and sell data
            create_excel_output(portfolio_buy, portfolio_sell, xlsx_path, category="Portfolio")
            create_pdf_report(portfolio_buy, portfolio_sell, pdf_path, timestamp, category="Portfolio")
            
            print(f"\n✓ Excel: {xlsx_path.name}")
            print(f"✓ PDF: {pdf_path.name}")
else:
    print("⚠️ No portfolio tickers found in stocks.txt")

## Scan Portfolio Stocks

Quick scan of your 9 portfolio stocks from stocks.txt. Shows both buy opportunities and sell signals.

In [7]:
print("Fetching S&P 500 ticker list...")
sp500_tickers = get_sp500_tickers()
print(f"Found {len(sp500_tickers)} S&P 500 stocks\n")

# Run scan
sp500_results = scan_stocks(
    sp500_tickers,
    category="S&P 500",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not sp500_results.empty:
    # Get buy signals (all and quality-filtered)
    sp500_buy_all = filter_buy_signals(sp500_results, 'FULL HOLD + ADD', quality_filter=False)
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD', quality_filter=True)
    
    # Get sell signals (all bearish signals, matching portfolio_analysis logic)
    sp500_sell = filter_sell_signals(sp500_results, quality_filter=False)
    
    # Display summary
    print(f"\n📊 S&P 500 Results:")
    print(f"   ✅ {len(sp500_buy)} buy signals (EXCELLENT/GOOD/OK)")
    if len(sp500_buy_all) > len(sp500_buy):
        print(f"   ⏸️  {len(sp500_buy_all) - len(sp500_buy)} extended/weak (wait)")
    print(f"   🔴 {len(sp500_sell)} sell signals (bearish)")
    
    # Display buy opportunities
    if not sp500_buy.empty:
        print("\n✅ TOP BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
        display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
        if all(col in sp500_buy.columns for col in display_cols):
            print(sp500_buy[display_cols].head(10).to_string(index=False))
        else:
            print(sp500_buy[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Display sell signals (matches portfolio_analysis behavior)
    if not sp500_sell.empty:
        print("\n🔴 SELL SIGNALS (Bearish - reduce exposure):")
        display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
        if all(col in sp500_sell.columns for col in display_cols):
            print(sp500_sell[display_cols].head(10).to_string(index=False))
        else:
            print(sp500_sell[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Save combined results if we have either buy or sell signals
    if not sp500_buy.empty or not sp500_sell.empty:
        xlsx_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_sp500_{timestamp}.pdf'
        
        # Create reports with both buy and sell data
        create_excel_output(sp500_buy, sp500_sell, xlsx_path, category="S&P 500")
        create_pdf_report(sp500_buy, sp500_sell, pdf_path, timestamp, category="S&P 500")
        
        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")
else:
    print("⚠️ No results from S&P 500 scan")

Fetching S&P 500 ticker list...
[OK] Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

[SCAN] Scanning 503 S&P 500 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [4/503] ABBV   -> FULL HOLD + ADD      $224.12 | Quality: EXTENDED   ⚠
[OK] [9/503] AFL    -> FULL HOLD + ADD      $110.16 | Quality: OK         ✓
[OK] [14/503] ALB    -> FULL HOLD + ADD      $157.90 | Quality: EXTENDED   ⚠
[OK] [19/503] GOOGL  -> FULL HOLD + ADD      $326.54 | Quality: EXTENDED   ⚠
[OK] [20/503] ALL    -> FULL HOLD + ADD      $209.35 | Quality: EXTENDED   ⚠
[OK] [21/503] GOOG   -> FULL HOLD + ADD      $326.96 | Quality: EXTENDED   ⚠
[OK] [23/503] AMZN   -> FULL HOLD + ADD      $245.58 | Quality: EXTENDED   ⚠
[OK] [27/503] AXP    -> FULL HOLD + ADD      $383.70 | Quality: EXTENDED   ⚠
[OK] [28/503] AIG    -> FULL HOLD + ADD      $78.30 | Quality: CAUTION    ⚠
[OK] [32/503] AME    -> FULL HOLD + ADD      $211.26 | Quality: EXTENDED   ⚠
[OK] [33/503] AMGN   

## Scan NASDAQ 100

Scan ~100 tech-heavy stocks. Takes 2-5 minutes. Shows both buy opportunities (FULL HOLD + ADD) and sell signals (bearish signals).

In [8]:
print("Fetching NASDAQ 100 ticker list...")
nasdaq100_tickers = get_nasdaq100_tickers()
print(f"Found {len(nasdaq100_tickers)} NASDAQ 100 stocks\n")

# Run scan
nasdaq100_results = scan_stocks(
    nasdaq100_tickers,
    category="NASDAQ 100",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not nasdaq100_results.empty:
    # Get buy signals (all and quality-filtered)
    nasdaq100_buy_all = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD', quality_filter=False)
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD', quality_filter=True)
    
    # Get sell signals (all bearish signals, matching portfolio_analysis logic)
    nasdaq100_sell = filter_sell_signals(nasdaq100_results, quality_filter=False)
    
    # Display summary
    print(f"\n📊 NASDAQ 100 Results:")
    print(f"   ✅ {len(nasdaq100_buy)} buy signals (EXCELLENT/GOOD/OK)")
    if len(nasdaq100_buy_all) > len(nasdaq100_buy):
        print(f"   ⏸️  {len(nasdaq100_buy_all) - len(nasdaq100_buy)} extended/weak (wait)")
    print(f"   🔴 {len(nasdaq100_sell)} sell signals (bearish)")
    
    # Display buy opportunities
    if not nasdaq100_buy.empty:
        print("\n✅ TOP BUY OPPORTUNITIES (EXCELLENT/GOOD/OK quality):")
        display_cols = ['ticker', 'signal', 'current_price', 'buy_quality', 's1']
        if all(col in nasdaq100_buy.columns for col in display_cols):
            print(nasdaq100_buy[display_cols].head(10).to_string(index=False))
        else:
            print(nasdaq100_buy[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Display sell signals (matches portfolio_analysis behavior)
    if not nasdaq100_sell.empty:
        print("\n🔴 SELL SIGNALS (Bearish - reduce exposure):")
        display_cols = ['ticker', 'signal', 'current_price', 'r1_quality', 'r1']
        if all(col in nasdaq100_sell.columns for col in display_cols):
            print(nasdaq100_sell[display_cols].head(10).to_string(index=False))
        else:
            print(nasdaq100_sell[['ticker', 'signal', 'current_price']].head(10).to_string(index=False))
    
    # Save combined results if we have either buy or sell signals
    if not nasdaq100_buy.empty or not nasdaq100_sell.empty:
        xlsx_path = results_dir / f'nasdaq100_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_nasdaq100_{timestamp}.pdf'
        
        # Create reports with both buy and sell data
        create_excel_output(nasdaq100_buy, nasdaq100_sell, xlsx_path, category="NASDAQ 100")
        create_pdf_report(nasdaq100_buy, nasdaq100_sell, pdf_path, timestamp, category="NASDAQ 100")
        
        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")
else:
    print("⚠️ No results from NASDAQ 100 scan")

Fetching NASDAQ 100 ticker list...
[OK] Loaded 101 NASDAQ 100 tickers

Found 101 NASDAQ 100 stocks

[SCAN] Scanning 101 NASDAQ 100 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
[OK] [5/101] GOOGL  -> FULL HOLD + ADD      $326.54 | Quality: EXTENDED   ⚠
[OK] [6/101] GOOG   -> FULL HOLD + ADD      $326.96 | Quality: EXTENDED   ⚠
[OK] [7/101] AMZN   -> FULL HOLD + ADD      $245.58 | Quality: EXTENDED   ⚠
[OK] [9/101] AMGN   -> FULL HOLD + ADD      $331.40 | Quality: GOOD       ✓
[OK] [10/101] ADI    -> FULL HOLD + ADD      $297.67 | Quality: EXTENDED   ⚠
[OK] [11/101] AAPL   -> FULL HOLD + ADD      $256.99 | Quality: EXTENDED   ⚠
[OK] [12/101] AMAT   -> FULL HOLD + ADD      $282.45 | Quality: EXTENDED   ⚠
[OK] [13/101] APP    -> FULL HOLD + ADD      $608.29 | Quality: EXTENDED   ⚠
[OK] [15/101] ASML   -> FULL HOLD + ADD      $1,189.92 | Quality: EXTENDED   ⚠
[OK] [16/101] AZN    -> FULL HOLD + ADD      $94.57 | Quality: EXTENDED   ⚠
[OK] [22/